In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from tensorflow.keras.models import Sequential
from sklearn.compose import ColumnTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/dlsp-portfolio-starter-code/cover_data.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded file from: {file_path}")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure the path is correct and the file exists in your Google Drive.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully loaded file from: /content/drive/MyDrive/dlsp-portfolio-starter-code/cover_data.csv


In [44]:
features = df.iloc[:, 0:-1]
labels = df.iloc[:, len(df.columns) - 1:len(df.columns)]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

#labels must be indexed at 0 for sparse categorical cross entropy
y_train = y_train - 1
y_test = y_test - 1

ct = ColumnTransformer([("scaler", StandardScaler(), X_train.columns)])
ct.fit_transform(X_train)
ct.transform(X_test)

array([[ 0.89452564,  0.3230179 , -0.54821295, ..., -0.16570694,
        -0.156514  , -0.12434232],
       [-1.23385746,  0.18013481, -0.68177483, ..., -0.16570694,
        -0.156514  , -0.12434232],
       [-1.16957743,  1.43929201, -0.41465106, ..., -0.16570694,
        -0.156514  , -0.12434232],
       ...,
       [ 1.00165901, -0.09670117,  1.72233914, ..., -0.16570694,
        -0.156514  , -0.12434232],
       [ 0.7981056 ,  0.90348044, -0.14752728, ..., -0.16570694,
        -0.156514  , -0.12434232],
       [-0.00896585,  1.29640892, -0.54821295, ..., -0.16570694,
        -0.156514  , -0.12434232]])

In [59]:
model = Sequential()
input_shape = X_train.shape[1]
model.add(Dense(units=128, activation='relu', input_shape=(input_shape,)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=8, activation='relu'))
num_classes = y_train['class'].nunique()
model.add(Dense(units=num_classes, activation='softmax'))
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                │ (None, 128)            │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 8)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 7)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,703 (69.15 KB)

 Trainable params: 17,703 (69.15 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric.")

Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric.


In [63]:
history = model.fit(X_train, y_train, epochs=100, batch_size=1024, validation_data=(X_test, y_test), callbacks=EarlyStopping(min_delta=0.001, patience=3, restore_best_weights=True))
print("Model training complete. Training history stored in 'history' variable.")

Epoch 1/100
454/454 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4881 - loss: 1.2036 - val_accuracy: 0.4876 - val_loss: 1.2051
Epoch 2/100
454/454 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4881 - loss: 1.2046 - val_accuracy: 0.4876 - val_loss: 1.2052
Epoch 3/100
454/454 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4867 - loss: 1.2079 - val_accuracy: 0.4876 - val_loss: 1.2052
Epoch 4/100
454/454 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4884 - loss: 1.2044 - val_accuracy: 0.4876 - val_loss: 1.2052
Model training complete. Training history stored in 'history' variable.
